#### [LangChain Handbook](https://pinecone.io/learn/langchain)

# LangChain Expression Language (LCEL)

The **L**ang**C**hain **E**xpression **L**anguage (LCEL) is a abstraction of some interesting Python concepts into a format that enables a "minimalist" code layer for building chains of LangChain components.

LCEL comes with strong support for:

1. Superfast development of chains.
2. Advanced features such as streaming, async, parallel execution, and more.
3. Easy integration with LangSmith and LangServe.

In [1]:
#!pip uninstall -y langchain langchain-core langchain-community langchain-cohere langchain_chroma

In [7]:
!pip install langchain-core langchain langchain-community langchain-cohere langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.3/390.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 38.4 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.5
    Uninstalling openai-1.54.5:
      Successfully uninstalled openai-1.54.5


## LCEL Syntax

To understand LCEL syntax let's first build a simple chain in typical Python syntax.

In [8]:
# from langchain.chat_models import ChatAnthropic
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain_openai import ChatOpenAI
import os

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
os.environ["OPENAI_API_KEY"] = open('/content/drive/MyDrive/GenAI classes/Gen AI tobeshared/Demo Notebooks/key.txt','r').read()

In [11]:
# Creating an instance of ChatOpenAI with the model set to 'gpt-4-turbo'
llm = ChatOpenAI(model='gpt-4-turbo')

In [12]:
# ANTHROPIC_API_KEY = "<<YOUR_ANTHROPIC_API_KEY>>"

# if ANTHROPIC_API_KEY == "<<YOUR_ANTHROPIC_API_KEY>>":
#     raise ValueError("Please set your ANTHROPIC_API_KEY")


prompt = ChatPromptTemplate.from_template(
    "Give me small report about {topic}"
)

# model = ChatAnthropic(
#     model="claude-2.1",
#     max_tokens_to_sample=512,
#     anthropic_api_key=ANTHROPIC_API_KEY
# )  # swap Anthropic for OpenAI with `ChatOpenAI` and `openai_api_key`

output_parser = StrOutputParser()

In [13]:
lcel_chain = prompt | llm | output_parser

# and run
out = lcel_chain.invoke({"topic": "Artificial Intelligence"})
print(out)

**Artificial Intelligence (AI): An Overview**

**Definition:**
Artificial Intelligence (AI) refers to the simulation of human intelligence in machines that are programmed to think like humans and mimic their actions. The term may also be applied to any machine that exhibits traits associated with a human mind such as learning and problem-solving.

**History and Development:**
The concept of AI dates back to antiquity with myths, stories, and rumors of artificial beings endowed with intelligence or consciousness by master craftsmen. The field of AI research was formally founded at a conference on the campus of Dartmouth College in the summer of 1956. Since then, AI has evolved and experienced both high expectations and disappointing setbacks.

**Key Components:**
1. **Machine Learning (ML)**: This involves algorithms that allow software to improve its performance on tasks through experience. ML is one of the most common tools used in AI.
2. **Neural Networks**: Inspired by the human bra

Pretty cool, the way that this pipe operator is being used is that it is taking output from the function to the _left_ and feeding it into the function on the _right_.

## How the Pipe Operator Works

To really understand LCEL we can take a look at how this pipe operation works. We know it takes output from the _right_ and feeds it to the _left_ — but this isn't typical Python, so how is this implemented? Let's try creating our own version of this with some simple functions.

We will be using the `__or__` method within Python class objects. When we place two classes together like `chain = class_a | class_b` the Python interpreter will check if these classes contain this `__or__` method. If they do then our `|` code will be translated into `chain = class_a.__or__(class_b)`.

That means both of these patterns will return the same thing:

```python
# object approach
chain = class_a.__or__(class_b)
chain("some input")

# pipe approach
chain = class_a | class_b
chain("some input")

```

With that in mind, we can build a `Runnable` class that consumes a function and turns it into a function that can be chained with other functions using the pipe operator `|`.

In [14]:
class Runnable:
    def __init__(self, func):
        self.func = func

    def __or__(self, other):
        def chained_func(*args, **kwargs):
            # the other func consumes the result of this func
            return other(self.func(*args, **kwargs))
        return Runnable(chained_func)

    def __call__(self, *args, **kwargs):
        return self.func(*args, **kwargs)

Let's implement this to take the value `3`, add `5` (giving `8`), and multiply by `2` (giving `16`).

> Add blockquote



In [15]:
def add_five(x):
    return x + 5

def multiply_by_two(x):
    return x * 2

# wrap the functions with Runnable
add_five = Runnable(add_five)
multiply_by_two = Runnable(multiply_by_two)

# run them using the object approach
chain = add_five.__or__(multiply_by_two)
chain(3)  # should return 16

16

Using `__or__` directly we get the correct answer, now let's try using the pipe operator `|` to chain them together:

In [16]:
# chain the runnable functions together
chain = add_five | multiply_by_two

# invoke the chain
chain(3)  # we should return 16

16

Using either method we get the same response, at it's core this is what LCEL is doing, but there is more.

## LCEL Deep Dive

Now that we understand what this syntax is doing under the hood, let's explore it within the context of LCEL and see a few of the additional methods that LangChain has provided to maximize flexibility when working with LCEL.

### Runnables

When working with LCEL we may find that we need to modify the structure or values being passed between components — for this we can use _runnables_. Let's try:

In [ ]:
# pip install langchain_community
!# pip install -U langchain-cohere

In [18]:
# pip install langchain_chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 72.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.6/166.6 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 3.8 MB/s eta 0:00:

In [19]:
# from langchain.embeddings import CohereEmbeddings
# from langchain.vectorstores import DocArrayInMemorySearch
import cohere
from langchain_chroma import Chroma


api_key = '6HMOVH4w6xuqDG6DMJiTMfCIqIcVUl8AIT2elyLf'


# embedding = CohereEmbeddings(
#     model="embed-english-light-v3.0",
#     cohere_api_key=api_key
# )

import langchain_cohere
from langchain_cohere import CohereEmbeddings

embedding = CohereEmbeddings(
    model="embed-english-light-v3.0",
    cohere_api_key=api_key
)

# vecstore_a = DocArrayInMemorySearch.from_texts(
#     ["half the info will be here", "James' birthday is the 7th December"],
#     embedding=embedding,

# )
# vecstore_b = DocArrayInMemorySearch.from_texts(
#     ["and half here", "James was born in 1994"],
#     embedding=embedding
# )

In [20]:
from langchain.docstore.document import Document

docs_a = [Document(page_content=doc) for doc in ["half the info will be here", "James' birthday is the 7th December"]]
docs_b = [Document(page_content=doc) for doc in ["and half here", "James was born in 1994"]]

In [21]:
chroma_db_a = Chroma.from_documents(documents=docs_a,
                                  embedding=embedding)
chroma_db_b = Chroma.from_documents(documents=docs_b, collection_name='wiki_db',
                                  embedding=embedding)



```
# This is formatted as code
```

First let's try passing a question through to a single one of these `vecstore` objects:

In [22]:
from langchain_core.runnables import (
    RunnableParallel,
    RunnablePassthrough
)

retriever_a = chroma_db_a.as_retriever(search_kwargs={"k": min(4, len(docs_a))})
retriever_b = chroma_db_b.as_retriever(search_kwargs={"k": min(4, len(docs_a))})

prompt_str = """Answer the question below using the context:

Context: {context}

Question: {question}

Answer: """
prompt = ChatPromptTemplate.from_template(prompt_str)

retrieval = RunnableParallel(
    {"context": retriever_a, "question": RunnablePassthrough()}
)

chain = retrieval | prompt | llm | output_parser

In [23]:
out = chain.invoke("when was James born?")
print(out)

James was born on the 7th of December.


Here we have used `RunnableParallel` to create two parallel streams of information, one for `"context"` that is information fed in from `retriever_a`, and another for `"question"` which is the _passthrough_ information, ie the information that is passed through from our `chain.invoke("when was James born?")` call.

Using this information the chain is close to answering the question but it doesn't have enough information, it is missing the information that we have stored in `retriever_b`. Fortunately, we can have multiple parallel information streams using the `RunnableParallel` object.

In [24]:
prompt_str = """Answer the question below using the context:

Context:
{context_a}
{context_b}

Question: {question}

Answer: """
prompt = ChatPromptTemplate.from_template(prompt_str)

retrieval = RunnableParallel(
    {
        "context_a": retriever_a, "context_b": retriever_b,
        "question": RunnablePassthrough()
    }
)

chain = retrieval | prompt | llm | output_parser

In [25]:
out = chain.invoke("when was James born?")
print(out)

James was born on the 7th December 1994.


Now the `RunnableParallel` object is allowing us to search with `retriever_a` _and_ `retriever_b` in parallel, ie at the same time.

## Runnable Lambdas

The `RunnableLambda` is a LangChain abstraction that allows us to turn Python functions into pipe-compatible function, similar to the `Runnable` class we created near the beginning of this notebook.

Let's try it out with our earlier `add_five` and `multiply_by_two` functions.

In [26]:
from langchain_core.runnables import RunnableLambda

# wrap the functions with RunnableLambda
add_five = RunnableLambda(add_five)
multiply_by_two = RunnableLambda(multiply_by_two)

As with our earlier `Runnable` abstraction, we can use `|` operators to chain together our `RunnableLambda` abstractions.

In [27]:
chain = add_five | multiply_by_two

Unlike our `Runnable` abstraction, we cannot run the `RunnableLambda` chain by calling it directly, instead we must call `chain.invoke`:

In [28]:
chain.invoke(3)

16

As before, we can see the same answer. Naturally we can feed custom functions into our chains using this approach. Let's try a short chain and see where we might want to insert a custom function:

In [29]:
prompt_str = "Tell me an short fact about {topic}"
prompt = ChatPromptTemplate.from_template(prompt_str)

chain = prompt | llm | output_parser

In [30]:
chain.invoke({"topic": "Artificial Intelligence"})

'Artificial Intelligence (AI) refers to systems or machines that mimic human intelligence to perform tasks and can iteratively improve themselves based on the information they collect. AI manifests in various forms, such as chatbots, robots, autonomous vehicles, and applications like voice assistants and image recognition software.'

In [31]:
chain.invoke({"topic": "Artificial Intelligence"})

'Artificial intelligence (AI) can learn and adapt over time. Machine learning, a subset of AI, allows computers to improve their performance on tasks through experience, without being explicitly programmed for every circumstance.'

The returned text always includes the initial `"Here's a short fact about ...\n\n"` — let's add a function to split on double newlines `"\n\n"` and only return the fact itself.

In [33]:
def extract_fact(x):
    if "\n\n" in x:
        return "\n".join(x.split("\n\n")[1:])
    else:
        return x

get_fact = RunnableLambda(extract_fact)

chain = prompt | llm | output_parser | get_fact

In [32]:
chain.invoke({"topic": "Artificial Intelligence"})

'Artificial intelligence (AI) is a branch of computer science that aims to create machines capable of performing tasks that typically require human intelligence, such as visual perception, speech recognition, decision-making, and language translation.'

In [34]:
chain.invoke({"topic": "Artificial Intelligence"})

'Artificial Intelligence (AI) is capable of learning and adapting. Through techniques such as machine learning, AI systems can analyze large amounts of data, learn patterns, and make decisions with minimal human intervention.'

Now we're getting well formatted outputs using our final custom `get_fact` function.

---

That covers the essentials you need to getting started and building with the **L**ang**C**hain **E**xpression **L**anguage (LCEL). With it, we can put together chains very easily — and the current focus of the LangChain team is on further LCEL development and support.

The pros and cons of LCEL are varied. Those that love it tend to focus on the minimalist code style, LCEL's support for streaming, parallel operations, and async, and LCEL's nice integration with LangChain's focus on chaining components together.

There are also people that are less fond of LCEL. Typically people will point to it being yet another abstraction on top of an already very abstract library, that the syntax is confusing, against [the Zen of Python](https://peps.python.org/pep-0020/), and requires too much effort to learn new (or uncommon) syntax.

Both viewpoints are entirely valid, LCEL is a very different approach — ofcourse there are major pros and major cons. In either case, if you're willing to spend some time learning the syntax, it allows us to develop very quickly, and with that in mind it's well worth learning.

---